In [1]:
# Imports & helpers
import random
import math
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Markdown
import ipywidgets as widgets
import numpy as np

# For consistent visuals
plt.rcParams["figure.figsize"] = (7,5)

In [8]:
# Scatter Plot Game (Click Version)

def scatter_plot_game(num_points=8, coord_range=10, tolerance=0.75):
    # Player clicks on the graph to identify the coordinates of a randomly selected point.
    # Generate integer points
    points = [(random.randint(-coord_range, coord_range),
               random.randint(-coord_range, coord_range))
              for _ in range(num_points)]

    target_index = random.randint(1, num_points)
    tx, ty = points[target_index - 1]

    fig, ax = plt.subplots()
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]

    ax.scatter(xs, ys, s=80)
    for i, (x, y) in enumerate(points, start=1):
        ax.text(x + 0.1, y + 0.1, str(i), fontsize=10)

    ax.axhline(0, color="lightgray")
    ax.axvline(0, color="lightgray")
    ax.set_xlim(-coord_range - 1, coord_range + 1)
    ax.set_ylim(-coord_range - 1, coord_range + 1)
    ax.set_title(f"Click on point number {target_index}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    def on_click(event):
        if event.xdata is None or event.ydata is None:
            return

        cx, cy = event.xdata, event.ydata
        dist = math.dist((cx, cy), (tx, ty))

        ax.plot(cx, cy, "rx", markersize=12)
        fig.canvas.draw()

        if dist <= tolerance:
            print(f"Correct! You clicked near ({tx}, {ty}) 🎯")
        else:
            print(f"Incorrect. That was ({cx:.2f}, {cy:.2f})")
            print(f"Correct coordinates were ({tx}, {ty})")

        fig.canvas.mpl_disconnect(cid)

    cid = fig.canvas.mpl_connect("button_press_event", on_click)
    plt.show()

In [3]:
# Algebra Practice Game
def generate_one_step():
    # Form: x + b = c  or a*x = c  (use integer solutions)
    if random.random() < 0.5:
        # Addition or subtraction
        x = random.randint(-12, 12)
        b = random.randint(-12, 12)
        c = x + b
        eq = f"x + ({b}) = {c}" if b >=0 else f"x - ({-b}) = {c}"
        return eq, x
    else:
        # Multiplication or division (avoid zero divisor)
        a = random.choice([i for i in range(-8,9) if i not in (0,1,-1)])
        x = random.randint(-12, 12)
        c = a * x
        eq = f"{a}*x = {c}"
        return eq, x

def generate_two_step():
    # Form: a*x + b = c  (a != 0)
    a = random.choice([i for i in range(-9,10) if i != 0])
    x = random.randint(-10,10)
    b = random.randint(-12,12)
    c = a*x + b
    eq = f"{a}*x + ({b}) = {c}" if b>=0 else f"{a}*x - ({-b}) = {c}"
    return eq, x

def algebra_practice(rounds=8, mix=True):
    score = 0
    for i in range(1, rounds+1):
        if mix:
            if random.random() < 0.5:
                eq, sol = generate_one_step()
            else:
                eq, sol = generate_two_step()
        else:
            eq, sol = generate_two_step()
        print(f"Problem {i}:  Solve for x :  {eq}")
        ans = input("Your answer (integer): ").strip()
        try:
            user_ans = int(ans)
        except:
            print("Invalid integer input. Marked incorrect.")
            continue
        if user_ans == sol:
            print("Correct!\n")
            score += 1
        else:
            print(f"Incorrect. Correct answer: {sol}\n")
    print(f"Done. Score: {score}/{rounds}")
    return score

print("Algebra Practice ready. To play: algebra_practice(rounds=8, mix=True)")

Algebra Practice ready. To play: algebra_practice(rounds=8, mix=True)


In [4]:
# Projectile Game - Level 1 (sliders)
def projectile_trajectory(angle_deg, velocity, steps=300, g=9.8):
    angle = math.radians(angle_deg)
    # Parametric in t
    t_flight = 2 * velocity * math.sin(angle) / g
    t_vals = np.linspace(0, max(t_flight*1.1, 0.1), steps)
    x = velocity * np.cos(angle) * t_vals
    y = velocity * np.sin(angle) * t_vals - 0.5 * g * t_vals**2
    return x, y

def projectile_game_with_sliders():
    # Random wall parameters
    wall_x = random.uniform(15, 40)  # Horizontal distance where wall stands
    wall_w = random.uniform(1.5, 4.0)
    wall_h = random.uniform(3, 18)

    out = widgets.Output()

    angle_slider = widgets.FloatSlider(value=45, min=10, max=80, step=1, description="Angle°")
    velocity_slider = widgets.FloatSlider(value=35, min=5, max=80, step=1, description="Velocity")
    fire_button = widgets.Button(description="Launch!")
    reset_button = widgets.Button(description="New Wall")
    info = widgets.HTML(value=f"<b>Wall at x≈{wall_x:.1f}, width≈{wall_w:.1f}, height≈{wall_h:.1f}</b>")

    def draw(angle, velocity):
        x, y = projectile_trajectory(angle, velocity)
        with out:
            clear_output(wait=True)
            fig, ax = plt.subplots()
            ax.plot(x, y)
            # Draw ground
            ax.axhline(0, color='saddlebrown')
            # Wall rectangle
            rect = plt.Rectangle((wall_x, 0), wall_w, wall_h, color='gray', alpha=0.7)
            ax.add_patch(rect)
            ax.set_xlim(0, max(wall_x + wall_w + 5, max(x)*0.9 + 1))
            ax.set_ylim(0, max(max(y)+5, wall_h+5))
            ax.set_title(f"Angle={angle:.1f}°, Velocity={velocity:.1f}")
            plt.show()

    def on_fire(b):
        angle = angle_slider.value
        velocity = velocity_slider.value
        x, y = projectile_trajectory(angle, velocity)
        # Compute y at center of wall (approx)
        test_x = wall_x + wall_w/2
        # Find y at that x by interpolation
        if max(x) < test_x:
            hit = False
            reason = "Projectile land before reaching the wall."
        else:
            yi = np.interp(test_x, x, y)
            if yi > wall_h:
                hit = True
                reason = f"Cleared! y at wall ≈ {yi:.2f} > wall height {wall_h:.2f}"
            else:
                hit = False
                reason = f"Hit wall. y at wall ≈ {yi:.2f} ≤ wall height {wall_h:.2f}"
        with out:
            clear_output(wait=True)
            draw(angle, velocity)
            display(Markdown(f"**Result:** {reason}"))

    def on_reset(b):
        # Redraw a new game by re-creating everything (quick approach)
        clear_output(wait=True)
        print("Refreshing cell to create a new wall. Re-run the cell to get a new random wall.")

    fire_button.on_click(on_fire)
    reset_button.on_click(on_reset)

    display(info)
    display(angle_slider, velocity_slider, widgets.HBox([fire_button, reset_button]), out)
    draw(angle_slider.value, velocity_slider.value)

print("Projectile (sliders) ready. Run: projectile_game_with_sliders()")

Projectile (sliders) ready. Run: projectile_game_with_sliders()


In [5]:
# Projectile Game - Level 2 (enter a,b,c parabola)
def parabola_game():
    # Player enters coefficients a, b, c for y = a*x^2 + b*x + c. A random wall is generated; check whether the parabola clears the wall.
    wall_x = random.uniform(10, 35)
    wall_w = random.uniform(1.5, 4.5)
    wall_h = random.uniform(3, 18)
    print(f"Wall generated at x≈{wall_x:.2f}, width≈{wall_w:.2f}, height≈{wall_h:.2f}")
    a = float(input("Enter coefficient a: ").strip())
    b = float(input("Enter coefficient b: ").strip())
    c = float(input("Enter coefficient c: ").strip())

    xs = np.linspace(0, max(wall_x + wall_w + 5, 40), 400)
    ys = a*xs**2 + b*xs + c

    # Check y at center of wall
    test_x = wall_x + wall_w/2
    y_at_wall = a*test_x**2 + b*test_x + c
    clear = y_at_wall > wall_h

    # Plot
    fig, ax = plt.subplots()
    ax.plot(xs, ys)
    ax.axhline(0, color='saddlebrown')
    rect = plt.Rectangle((wall_x, 0), wall_w, wall_h, color='gray', alpha=0.7)
    ax.add_patch(rect)
    ax.set_xlim(0, max(xs))
    ax.set_ylim(min(min(ys)-5, -5), max(max(ys)+5, wall_h+5))
    ax.set_title(f"Parabola y = {a}x^2 + {b}x + {c}")
    plt.show()

    if clear:
        print(f"Success! y at wall center ≈ {y_at_wall:.2f} > wall height {wall_h:.2f}")
    else:
        print(f"Fail. y at wall center ≈ {y_at_wall:.2f} ≤ wall height {wall_h:.2f}")
    return clear

print("Parabola Level ready. Run: parabola_game()")


Parabola Level ready. Run: parabola_game()


In [10]:
# Simple text menu to run any game
def run_menu():
    print("Choose a game to run:")
    print("1 - Scatter Plot Game")
    print("2 - Algebra Practice Game")
    print("3 - Projectile Game (sliders)")
    print("4 - Projectile Game (enter a,b,c)")
    choice = input("Enter 1/2/3/4: ").strip()

    if choice == "1":
        n = input("Number of points (default 8): ").strip() or "8"
        r = input("Coordinate range (default 10): ").strip() or "10"
        try:
            scatter_plot_game(int(n), int(r))
        except Exception as e:
            print("Error:", e)

    elif choice == "2":
        rounds = input("Rounds (default 8): ").strip() or "8"
        algebra_practice(int(rounds))

    elif choice == "3":
        projectile_game_with_sliders()

    elif choice == "4":
        parabola_game()

    else:
        print("Invalid choice.")

In [12]:
run_menu()

Choose a game to run:
1 - Scatter Plot Game
2 - Algebra Practice Game
3 - Projectile Game (sliders)
4 - Projectile Game (enter a,b,c)
Enter 1/2/3/4: 2
Rounds (default 8): 8
Problem 1:  Solve for x :  x - (2) = 6
Your answer (integer): 8
Correct!

Problem 2:  Solve for x :  5*x + (5) = 5
Your answer (integer): 0
Correct!

Problem 3:  Solve for x :  x + (3) = -2
Your answer (integer): -5
Correct!

Problem 4:  Solve for x :  x - (4) = -11
Your answer (integer): -7
Correct!

Problem 5:  Solve for x :  7*x = -35
Your answer (integer): -5
Correct!

Problem 6:  Solve for x :  -6*x = -54
Your answer (integer): 9
Correct!

Problem 7:  Solve for x :  x + (4) = -1
Your answer (integer): -5
Correct!

Problem 8:  Solve for x :  x - (10) = -17
Your answer (integer): -27
Incorrect. Correct answer: -7

Done. Score: 7/8
